<a href="https://colab.research.google.com/github/TaherBenAfia/Soil_Prediction/blob/main/best_score_1046.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [29]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.multioutput import MultiOutputRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error

import warnings
warnings.filterwarnings('ignore')

# **Importing Datasets**

In [30]:
# Load datasets
# Install dependencies as needed:
# pip install kagglehub[pandas-datasets]
import kagglehub
from kagglehub import KaggleDatasetAdapter

# Set the path to the file you'd like to load
file_path = "Train.csv"

# Load the latest version
df1 = kagglehub.load_dataset(
  KaggleDatasetAdapter.PANDAS,
  "muhammadqasimshabbir/amini-soil-prediction-challenge-dataset",
  file_path,
)
file_path1 = "Test.csv"

# Load the latest version
df2 = kagglehub.load_dataset(
  KaggleDatasetAdapter.PANDAS,
  "muhammadqasimshabbir/amini-soil-prediction-challenge-dataset",
  file_path1,
)
file_path2 = "Gap_Train.csv"

# Load the latest version
df3 = kagglehub.load_dataset(
  KaggleDatasetAdapter.PANDAS,
  "muhammadqasimshabbir/amini-soil-prediction-challenge-dataset",
  file_path2,
)
file_path3 = "Gap_Test.csv"

# Load the latest version
df4 = kagglehub.load_dataset(
  KaggleDatasetAdapter.PANDAS,
  "muhammadqasimshabbir/amini-soil-prediction-challenge-dataset",
  file_path3,
)
file_path4 = "SampleSubmission.csv"

# Load the latest version
df5 = kagglehub.load_dataset(
  KaggleDatasetAdapter.PANDAS,
  "muhammadqasimshabbir/amini-soil-prediction-challenge-dataset",
  file_path4,
)
file_path5 = "Sentinel1_data.csv"

# Load the latest version
df6 = kagglehub.load_dataset(
  KaggleDatasetAdapter.PANDAS,
  "muhammadqasimshabbir/amini-soil-prediction-challenge-dataset",
  file_path5,
)
file_path6 = "Sentinel2_data.csv"

# Load the latest version
df7 = kagglehub.load_dataset(
  KaggleDatasetAdapter.PANDAS,
  "muhammadqasimshabbir/amini-soil-prediction-challenge-dataset",
  file_path6,
)
file_path7 = "LANDSAT8_data_updated.csv"

# Load the latest version
df8 = kagglehub.load_dataset(
  KaggleDatasetAdapter.PANDAS,
  "muhammadqasimshabbir/amini-soil-prediction-challenge-dataset",
  file_path7,
)
file_path8 = "MODIS_MOD09GA_data.csv"

# Load the latest version
df9 = kagglehub.load_dataset(
  KaggleDatasetAdapter.PANDAS,
  "muhammadqasimshabbir/amini-soil-prediction-challenge-dataset",
  file_path8,
)
file_path9 = "MODIS_MOD11A1_data.csv"

# Load the latest version
df10 = kagglehub.load_dataset(
  KaggleDatasetAdapter.PANDAS,
  "muhammadqasimshabbir/amini-soil-prediction-challenge-dataset",
  file_path9,
)
file_path10 = "MODIS_MOD16A2_data.csv"

# Load the latest version
df11 = kagglehub.load_dataset(
  KaggleDatasetAdapter.PANDAS,
  "muhammadqasimshabbir/amini-soil-prediction-challenge-dataset",
  file_path10,
)
file_path11 = "MODIS_MOD13Q1_data.csv"

# Load the latest version
df12 = kagglehub.load_dataset(
  KaggleDatasetAdapter.PANDAS,
  "muhammadqasimshabbir/amini-soil-prediction-challenge-dataset",
  file_path11,
)
train_df = df1.copy()
test_df = df2.copy()
train_gap_df = df3.copy()
test_gap_df = df4.copy()
sample_submission = df5.copy()
#Ca
landsat=df8.copy()
modis_1 = df9.copy()
modis_2 = df10.copy()
modis_3 = df11.copy()
modis_4 = df12.copy()
sentinel_1 = df6.copy()
sentinel_2 = df7.copy()
#N
landsat_1=df8.copy()
modis_11 = df9.copy()
modis_22 = df10.copy()
modis_33 = df11.copy()
modis_44 = df12.copy()
sentinel_11 = df6.copy()
sentinel_22 = df7.copy()
train_N=train_df.copy()
test_N=test_df.copy()
#Mg
train_Mg = train_df.copy()
test_Mg = test_df.copy()
landsat_2=df8.copy()
modis_111 = df9.copy()
modis_222 = df10.copy()
modis_333 = df11.copy()
modis_444 = df12.copy()
sentinel_111 = df6.copy()
sentinel_222 = df7.copy()

In [31]:
test_gap_df = pd.merge(test_gap_df, test_df[['PID', 'BulkDensity']], on='PID', how='left')

# **Handling Missing Values**

In [32]:
# prompt: input missing values in train_df and test_df with the mean, only do it for columns that have missing values

# Fill missing values with the mean for columns with missing values in train_df
for column in train_df.columns:
  if train_df[column].isnull().any():
    train_df[column].fillna(train_df[column].mean(), inplace=True)

# Fill missing values with the mean for columns with missing values in test_df
for column in test_df.columns:
  if test_df[column].isnull().any():
    test_df[column].fillna(test_df[column].mean(), inplace=True)


# **Choosing the Targets**

In [33]:
target_columns = [ 'P', 'K', 'S', 'Fe', 'Mn', 'Zn', 'Cu', 'B']

# **Excluding "Ca" from Targets (predicting it alone)**

In [34]:
Ca=train_df["Ca"]
Ca

,Ca
0,6830
1,1180
2,1890
3,6660
4,7340
...,...
7739,1878
7740,1317
7741,1671
7742,1507


In [35]:
train_df.drop(columns=["Ca"],inplace=True)
train_df.drop(columns=["N"],inplace=True)
train_df.drop(columns=["Mg"],inplace=True)

# **Excluding "N" from Targets ( Predicting it alone)**

In [36]:
N= train_N["N"]
N
Mg = train_Mg["Mg"]

In [37]:
train_N.drop(columns =["N"],inplace= True)
train_Mg.drop(columns =["Mg"],inplace= True)

In [38]:
train_df.columns

Index(['site', 'PID', 'lon', 'lat', 'pH', 'alb', 'bio1', 'bio12', 'bio15',
       'bio7', 'bp', 'cec20', 'dows', 'ecec20', 'hp20', 'ls', 'lstd', 'lstn',
       'mb1', 'mb2', 'mb3', 'mb7', 'mdem', 'para', 'parv', 'ph20', 'slope',
       'snd20', 'soc20', 'tim', 'wp', 'xhp20', 'BulkDensity', 'P', 'K', 'S',
       'Fe', 'Mn', 'Zn', 'Cu', 'B'],
      dtype='object')

In [39]:
all_columns = {
    'landsat_1': list(landsat_1.columns),
    'modis_11': list(modis_11.columns),
    'modis_22': list(modis_22.columns),
    'modis_33': list(modis_33.columns),
    'modis_44': list(modis_44.columns),
    'sentinel_11': list(sentinel_11.columns),
    'sentinel_22': list(sentinel_22.columns),
    "train_N":list(train_N.columns),
}

for name, cols in all_columns.items():
    print(f"{name} columns ({len(cols)}): {cols}")


landsat_1 columns (14): ['QA_PIXEL', 'QA_RADSAT', 'SR_B1', 'SR_B2', 'SR_B3', 'SR_B4', 'SR_B5', 'SR_B6', 'SR_B7', 'ST_B10', 'date', 'lat', 'lon', 'PID']
modis_11 columns (11): ['date', 'lat', 'lon', 'sur_refl_b01', 'sur_refl_b02', 'sur_refl_b03', 'sur_refl_b04', 'sur_refl_b05', 'sur_refl_b06', 'sur_refl_b07', 'PID']
modis_22 columns (6): ['LST_Day_1km', 'LST_Night_1km', 'date', 'lat', 'lon', 'PID']
modis_33 columns (6): ['ET', 'PET', 'date', 'lat', 'lon', 'PID']
modis_44 columns (13): ['EVI', 'NDVI', 'RelativeAzimuth', 'SolarZenith', 'ViewZenith', 'date', 'lat', 'lon', 'sur_refl_b01', 'sur_refl_b02', 'sur_refl_b03', 'sur_refl_b07', 'PID']
sentinel_11 columns (9): ['VH', 'VV', 'date', 'instrumentMode', 'lat', 'lon', 'orbitProperties_pass', 'relativeOrbitNumber_start', 'PID']
sentinel_22 columns (21): ['B1', 'B11', 'B12', 'B2', 'B3', 'B4', 'B5', 'B6', 'B7', 'B8', 'B8A', 'B9', 'CLOUDY_PIXEL_PERCENTAGE', 'MEAN_SOLAR_ZENITH_ANGLE', 'NODATA_PIXEL_PERCENTAGE', 'SENSING_ORBIT_NUMBER', 'SPACECRA

# **Selecting the Most significant features for "N" from Satelite Data**

In [40]:
selected_satellite_features = [
    # Landsat
    'SR_B2', 'SR_B3', 'SR_B4', 'SR_B5', 'SR_B6', 'ST_B10',

    # MODIS 11
    'sur_refl_b01', 'sur_refl_b02', 'sur_refl_b03', 'sur_refl_b04',
    'sur_refl_b06', 'sur_refl_b07',

    # MODIS 22
    'LST_Day_1km', 'LST_Night_1km',

    # MODIS 33
    'ET', 'PET',

    # MODIS 44
    'NDVI', 'EVI', 'sur_refl_b01', 'sur_refl_b02', 'sur_refl_b03', 'sur_refl_b07',

    # Sentinel-1
    'VH', 'VV',

    # Sentinel-2
    'B3', 'B4', 'B5', 'B6', 'B7', 'B8', 'B8A', 'B11',
    'CLOUDY_PIXEL_PERCENTAGE',

    # Always keep for merging
    'date', 'PID'
]
selected_features = [
    # From landsat_1
    'SR_B2', 'SR_B3', 'SR_B4', 'SR_B5', 'SR_B6', 'SR_B7', 'ST_B10',  # Landsat bands related to vegetation, SWIR, thermal
    # From modis_11
    'sur_refl_b01', 'sur_refl_b02', 'sur_refl_b03', 'sur_refl_b07',  # MODIS bands (Red, NIR, SWIR)
    # From modis_22
    'LST_Day_1km', 'LST_Night_1km',  # Land Surface Temperature
    # From modis_33
    'ET', 'PET',  # Evapotranspiration, potentially tied to soil and plant moisture
    # From modis_44
    'NDVI', 'EVI',  # Vegetation indices
    # From sentinel_11
    'VH', 'VV',  # Radar backscatter, informative of surface roughness, moisture
    # From sentinel_22
    'date','B2', 'B3', 'B4', 'B5', 'B6', 'B7', 'B8', 'B8A', 'B11', 'B12',  # Key Sentinel-2 bands for vegetation, SWIR, NIR
    'CLOUDY_PIXEL_PERCENTAGE',  # May impact reliability of optical data
    # From train_N
    'PID','pH', 'alb', 'cec20', 'ecec20', 'ph20', 'xhp20', 'soc20', 'BulkDensity',  # Directly related to soil chemistry and texture
    'bio1', 'bio12', 'bio15', 'bio7',  # Bioclimatic variables relevant to soil/plant interaction
    'slope', 'ls', 'mdem'  # Terrain characteristics influencing nutrient flow
]



# **Selecting the Most Significant Features for "N" From main Data(Train_N)**

In [41]:
high_importance_soil_features = [
    'PID',
    'pH',         # soil acidity – strongly affects nitrogen form & availability
    'cec20',      # nutrient retention capacity – often correlates with total N
    'soc20',      # organic carbon – directly linked to nitrogen mineralization
    'BulkDensity',# affects soil porosity and root zone development
    'snd20',      # sand content – impacts leaching of nitrogen
    'ph20',       # subsurface pH (often more predictive than surface pH)
    'wp',         # wilting point – moisture condition affecting N cycling
    'ls',         # slope length – influences erosion/leaching of N
    'slope',      # terrain slope – similar effect
    'bio1',       # annual mean temperature – affects decomposition and N cycle
    'bio12',      # total annual precipitation – drives leaching & uptake
]


# **Dropping the Unselected Features (For "N" Prediction)**

In [42]:
# Landsat
landsat_1 = landsat_1[landsat_1.columns.intersection(selected_satellite_features)]

# MODIS datasets
modis_11 = modis_11[modis_11.columns.intersection(selected_satellite_features)]
modis_22 = modis_22[modis_22.columns.intersection(selected_satellite_features)]
modis_33 = modis_33[modis_33.columns.intersection(selected_satellite_features)]
modis_44 = modis_44[modis_44.columns.intersection(selected_satellite_features)]

# Sentinel datasets
sentinel_11 = sentinel_11[sentinel_11.columns.intersection(selected_satellite_features)]
sentinel_22 = sentinel_22[sentinel_22.columns.intersection(selected_satellite_features)]

# Soil dataset
train_N = train_N[train_N.columns.intersection(high_importance_soil_features)]

In [43]:
landsat_2 = landsat_2[landsat_2.columns.intersection(selected_features)]

# MODIS datasets
modis_111 = modis_111[modis_111.columns.intersection(selected_features)]
modis_222 = modis_222[modis_222.columns.intersection(selected_features)]
modis_333 = modis_333[modis_333.columns.intersection(selected_features)]
modis_444 = modis_444[modis_444.columns.intersection(selected_features)]

# Sentinel datasets
sentinel_111 = sentinel_111[sentinel_111.columns.intersection(selected_features)]
sentinel_222 = sentinel_222[sentinel_222.columns.intersection(selected_features)]

# Soil dataset
train_Mg = train_Mg[train_Mg.columns.intersection(selected_features)]

In [44]:
test_N = test_N[test_N.columns.intersection(high_importance_soil_features)]
test_Mg = test_Mg[test_Mg.columns.intersection(selected_features)]

# **Base Model (Targets excluding "N" and "Ca")**

In [45]:
model = MultiOutputRegressor(RandomForestRegressor(n_estimators=100, random_state=42,n_jobs=-1))

In [46]:
# Feature selection
X = train_df.drop(columns=target_columns)
y = train_df[target_columns]
X_test = test_df.drop(columns=['PID',"site"])

In [47]:
# Train-test split
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

In [48]:
X_train = X_train.drop(columns=['PID','site'])
X_val = X_val.drop(columns=['PID','site'])

In [49]:
model.fit(X_train, y_train)

MultiOutputRegressor(estimator=RandomForestRegressor(n_jobs=-1,
                                                     random_state=42))

In [50]:
y_pred = model.predict(X_val)

# **Evaluation**

In [51]:
# Evaluate model
mae = mean_absolute_error(y_val, y_pred)
mse = mean_squared_error(y_val, y_pred)
rmse = np.sqrt(mse)
print(f'MAE: {mae:.4f}, RMSE: {rmse:.4f}')

MAE: 26.3449, RMSE: 75.9286


In [52]:
test_predictions=model.predict(X_test)

In [53]:
# Split the predictions into separate columns
P_pred =  test_predictions[:, 0]  # Predictions for P
K_pred =  test_predictions[:, 1]  # Predictions for K
S_pred =  test_predictions[:, 2]  # Predictions for S
Fe_pred = test_predictions[:, 3]  # Predictions for Fe
Mn_pred = test_predictions[:, 4]  # Predictions for Mn
Zn_pred = test_predictions[:, 5]  # Predictions for Zn
Cu_pred = test_predictions[:, 6]  # Predictions for Cu
B_pred =  test_predictions[:, 7]  # Predictions for B


In [54]:
submission = pd.DataFrame({'PID': test_df['PID'], 'P': P_pred, 'K': K_pred, 'S': S_pred, 'Fe': Fe_pred, 'Mn': Mn_pred, 'Zn': Zn_pred, 'Cu': Cu_pred, 'B': B_pred})
submission.head()

,PID,P,K,S,Fe,Mn,Zn,Cu,B
0,ID_NGS9Bx,15.4269,179.27,10.5150,129.765,147.887,1.8131,4.5936,0.2113
1,ID_YdVKXw,8.9349,178.03,10.0201,119.243,130.930,1.5662,4.5469,0.2065
2,ID_MZAlfE,3.9915,186.44,8.8659,132.232,161.940,1.6959,4.4342,0.2356
3,ID_GwCCMN,1.7059,172.56,9.7395,136.026,162.780,1.8275,4.1322,0.2368
4,ID_K8sowf,7.4646,206.42,8.4816,122.090,128.877,2.0823,4.6589,0.2035


# **Dropping Unselected Features (For "Ca" Prediction)**

In [55]:
landsat.drop(columns=['QA_PIXEL', 'QA_RADSAT','lat','lon'],inplace=True)
modis_1.drop(columns=['lat','lon'],inplace=True)
modis_2.drop(columns=['lat','lon'],inplace=True)
modis_3.drop(columns=['lat','lon'],inplace=True)
modis_4.drop(columns=['lat','lon'],inplace=True)
sentinel_1.drop(columns=['lat','lon','instrumentMode',
       'orbitProperties_pass', 'relativeOrbitNumber_start'],inplace=True)
sentinel_2.drop(columns=['lat','lon','CLOUDY_PIXEL_PERCENTAGE','NODATA_PIXEL_PERCENTAGE','SENSING_ORBIT_NUMBER','SPACECRAFT_NAME'],inplace=True)
train_df.drop(columns=["lat",'lon','bio7', 'mb1', 'mb2', 'mb3', 'mb7', 'mdem', 'para', 'parv', 'tim', 'xhp20'],inplace=True)
test_df.drop(columns=["lat",'lon','bio7', 'mb1', 'mb2', 'mb3', 'mb7', 'mdem', 'para', 'parv', 'tim', 'xhp20'],inplace=True)

In [56]:
import pandas as pd

# Step 1: Convert 'date' to datetime and extract 'month'
def prepare_df(df):
    df['date'] = pd.to_datetime(df['date'])
    df['month'] = df['date'].dt.to_period('M')
    return df

# Step 2: Aggregate a satellite dataset by PID and month, then average over months
def aggregate_satellite(df, name_prefix):
    df = prepare_df(df)

    # Group by PID and month and calculate mean, max, min
    agg = df.groupby(['PID', 'month']).agg(['mean', 'max', 'min']).reset_index()

    # Flatten column names
    agg.columns = ['_'.join([col[0], col[1]]) if col[1] else col[0] for col in agg.columns.values]
    agg = agg.rename(columns={'PID_': 'PID', 'month_': 'month'})

    # Add prefix to features
    agg.columns = [f"{name_prefix}_{col}" if col not in ['PID', 'month'] else col for col in agg.columns]

    # Average over months
    agg = agg.groupby('PID').mean(numeric_only=True).reset_index()

    return agg

# Step 3: Impute missing values
def impute_missing(df):
    return df.fillna(df.mean(numeric_only=True))

# Step 4: Incremental merging and imputation
def incremental_merge(train_df, satellite_dfs_with_names):
    df = train_df.copy()

    for sat_df, prefix in satellite_dfs_with_names:
        print(f"Merging with: {prefix}")
        agg = aggregate_satellite(sat_df, prefix)
        df = df.merge(agg, on='PID', how='left')
        df = impute_missing(df)

    return df

# ---------- Usage ----------
# Pass list of (satellite_dataframe, prefix)
satellite_inputs = [
    (landsat, 'landsat'),
    (modis_1, 'modis1'),
    (modis_2, 'modis2'),
    (modis_3, 'modis3'),
    (modis_4, 'modis4'),
    (sentinel_1, 's1'),
    (sentinel_2, 's2'),
]
satellite_inputs_N=[
    (landsat_1, 'landsat'),
    (modis_11, 'modis1'),
    (modis_22, 'modis2'),
    (modis_33, 'modis3'),
    (modis_44, 'modis4'),
    (sentinel_11, 's1'),
    (sentinel_22, 's2'),
]
satellite_inputs_Mg=[
    (landsat_2, 'landsat'),
    (modis_111, 'modis1'),
    (modis_222, 'modis2'),
    (modis_333, 'modis3'),
    (modis_444, 'modis4'),
    (sentinel_111, 's1'),
    (sentinel_222, 's2'),
]

# Final merged and imputed dataframe
final_df = incremental_merge(train_df, satellite_inputs)
final_test_df=incremental_merge(test_df,satellite_inputs)
final_df_N=incremental_merge(train_N,satellite_inputs_N)
final_test_df_N=incremental_merge(test_N,satellite_inputs_N)
final_df_Mg=incremental_merge(train_Mg,satellite_inputs_Mg)
final_test_df_Mg=incremental_merge(test_Mg,satellite_inputs_Mg)

Merging with: landsat
Merging with: modis1
Merging with: modis2
Merging with: modis3
Merging with: modis4
Merging with: s1
Merging with: s2
Merging with: landsat
Merging with: modis1
Merging with: modis2
Merging with: modis3
Merging with: modis4
Merging with: s1
Merging with: s2
Merging with: landsat
Merging with: modis1
Merging with: modis2
Merging with: modis3
Merging with: modis4
Merging with: s1
Merging with: s2
Merging with: landsat
Merging with: modis1
Merging with: modis2
Merging with: modis3
Merging with: modis4
Merging with: s1
Merging with: s2
Merging with: landsat
Merging with: modis1
Merging with: modis2
Merging with: modis3
Merging with: modis4
Merging with: s1
Merging with: s2
Merging with: landsat
Merging with: modis1
Merging with: modis2
Merging with: modis3
Merging with: modis4
Merging with: s1
Merging with: s2


In [57]:
final_df["Ca"]=Ca
final_df_N["N"]=N
final_df_Mg["Mg"]=Mg

In [58]:
final_df["K"].nunique()

1072

In [59]:
final_df["Ca"].unique()


array([6830, 1180, 1890, ..., 1878, 1671, 1507])

In [60]:
final_df.shape

(7744, 159)

In [61]:
final_test_df.shape

(2418, 150)

In [62]:
from sklearn.model_selection import train_test_split
# Feature selection
X = final_df.drop(columns=['Ca'])
y = final_df[['Ca']]
# Train-test split
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)
X_train = X_train.drop(columns=['PID','site'])
X_val = X_val.drop(columns=['PID','site'])


In [63]:
from sklearn.model_selection import train_test_split
# Feature selection
X = final_df_N.drop(columns=['N'])
y = final_df_N[['N']]
X_test = final_test_df_N.drop(columns=['PID'])
# Train-test split
X_train_N, X_val_N, y_train_N, y_val_N = train_test_split(X, y, test_size=0.2, random_state=42)
X_train_N = X_train_N.drop(columns=['PID'])
X_val_N = X_val_N.drop(columns=['PID'])


In [64]:
from sklearn.model_selection import train_test_split
# Feature selection
X = final_df_Mg.drop(columns=['Mg'])
y = final_df_Mg[['Mg']]
# Train-test split
X_train_Mg, X_val_Mg, y_train_Mg, y_val_Mg = train_test_split(X, y, test_size=0.2, random_state=42)
X_train_Mg = X_train_Mg.drop(columns=['PID'])
X_val_Mg = X_val_Mg.drop(columns=['PID'])


# **Model for "N" Prediction**

In [65]:
model = RandomForestRegressor(n_estimators=1490, random_state=42,n_jobs=-1)

In [66]:
model.fit(X_train_N,y_train_N)

RandomForestRegressor(n_estimators=1490, n_jobs=-1, random_state=42)

In [67]:
X_test_Ca = final_test_df.drop(columns=['PID',"site"])
X_test_Mg = final_test_df.drop(columns=['PID',"site"])

# **model for "Ca" Prediction**

In [68]:
model1 = RandomForestRegressor(n_estimators=1490, random_state=42,n_jobs=-1)

In [69]:
X_train = X_train.drop(columns=target_columns)
X_val = X_val.drop(columns= target_columns)

In [70]:
model1.fit(X_train,y_train)

RandomForestRegressor(n_estimators=1490, n_jobs=-1, random_state=42)

In [71]:
print(X_train.columns.tolist())

['pH', 'alb', 'bio1', 'bio12', 'bio15', 'bp', 'cec20', 'dows', 'ecec20', 'hp20', 'ls', 'lstd', 'lstn', 'ph20', 'slope', 'snd20', 'soc20', 'wp', 'BulkDensity', 'landsat_SR_B1_mean', 'landsat_SR_B1_max', 'landsat_SR_B1_min', 'landsat_SR_B2_mean', 'landsat_SR_B2_max', 'landsat_SR_B2_min', 'landsat_SR_B3_mean', 'landsat_SR_B3_max', 'landsat_SR_B3_min', 'landsat_SR_B4_mean', 'landsat_SR_B4_max', 'landsat_SR_B4_min', 'landsat_SR_B5_mean', 'landsat_SR_B5_max', 'landsat_SR_B5_min', 'landsat_SR_B6_mean', 'landsat_SR_B6_max', 'landsat_SR_B6_min', 'landsat_SR_B7_mean', 'landsat_SR_B7_max', 'landsat_SR_B7_min', 'landsat_ST_B10_mean', 'landsat_ST_B10_max', 'landsat_ST_B10_min', 'modis1_sur_refl_b01_mean', 'modis1_sur_refl_b01_max', 'modis1_sur_refl_b01_min', 'modis1_sur_refl_b02_mean', 'modis1_sur_refl_b02_max', 'modis1_sur_refl_b02_min', 'modis1_sur_refl_b03_mean', 'modis1_sur_refl_b03_max', 'modis1_sur_refl_b03_min', 'modis1_sur_refl_b04_mean', 'modis1_sur_refl_b04_max', 'modis1_sur_refl_b04_min'

# ***Model for "Mg" Prediction***

In [72]:
model2 = RandomForestRegressor(n_estimators=1490, random_state=42,n_jobs=-1)

In [73]:
model2.fit(X_train_Mg,y_train_Mg)

RandomForestRegressor(n_estimators=1490, n_jobs=-1, random_state=42)

# **Evaluating "N" Prediction**

In [74]:
y_pred_N=model.predict(X_val_N)
mae = mean_absolute_error(y_val_N, y_pred_N)
mse = mean_squared_error(y_val_N, y_pred_N)
rmse = np.sqrt(mse)

print(f"MAE: {mae}")
print(f"RMSE: {rmse}")


MAE: 365.4357780078943
RMSE: 505.16306425281664


# **Evaluating "Ca" Prediction**

In [75]:
y_pred=model1.predict(X_val)
mae = mean_absolute_error(y_val, y_pred)
mse = mean_squared_error(y_val, y_pred)
rmse = np.sqrt(mse)

print(f"MAE: {mae}")
print(f"RMSE: {rmse}")


MAE: 1049.5377208071022
RMSE: 1547.4219708292107


# **Evaluating "Mg" Prediction**

In [76]:
y_pred_Mg=model2.predict(X_val_Mg)
mae = mean_absolute_error(y_val_Mg, y_pred_Mg)
mse = mean_squared_error(y_val_Mg, y_pred_Mg)
rmse = np.sqrt(mse)

print(f"MAE: {mae}")
print(f"RMSE: {rmse}")


MAE: 235.7062309088782
RMSE: 342.79748013313446


In [77]:
'''# Normalize the test features (using the same scaler as for training)
X_test_scaled = scaler.transform(X_test)  # transform, not fit_transform

# Predict using the trained model
prediction = best_model.predict(X_test_scaled)

# Since target variable was not scaled, DO NOT inverse_transform predictions
# Just flatten and add to submission
s = pd.Series(prediction.flatten(), name='Ca')
'''
# Predict using the model (returns 1D array for 'Ca')
prediction = model1.predict(X_test_Ca)
predicction_N = model.predict(X_test)
prediction_Mg = model2.predict(X_test_Mg)
# Add 'Ca' predictions to the submission DataFrame
submission['Ca'] = prediction
submission['N'] = predicction_N
submission['Mg'] = prediction_Mg

# Reorder columns as required
new_order = ['PID', 'N', 'P', 'K', 'Ca', 'Mg', 'S', 'Fe', 'Mn', 'Zn', 'Cu', 'B']
submission = submission[new_order]



ValueError: The feature names should match those that were passed during fit.
Feature names unseen at fit time:
- alb
- bio15
- bio7
- ecec20
- landsat_SR_B7_max
- ...
Feature names seen at fit time, yet now missing:
- modis1_sur_refl_b04_max
- modis1_sur_refl_b04_mean
- modis1_sur_refl_b04_min
- modis1_sur_refl_b06_max
- modis1_sur_refl_b06_mean
- ...


In [ ]:
submission.head()

In [ ]:
# prompt: turn submission into a 3 column file that has the column PID, Nutrient, Value

submission_melted = submission.melt(id_vars=['PID'], var_name='Nutrient', value_name='Available_Nutrients_in_ppm')
submission_melted = submission_melted.sort_values('PID')
submission_melted.head()

# Calculate the nutrient gaps

Transform your predicted ppm values to available nutrients (kg/ha) to determine the nutrient gaps.

We are determining the nutrients needed for Maize crops to yeild 4 tons/ha at a depth of 20cm.

Available nutrients are calculated by

`Available (kg/ha) = ppm × soil depth (cm) × bulk density (g/cm³) × 0.1`

In [ ]:
# prompt: merge test_gap_df with submission_melted on PID and Nutrient
nutrient_df = pd.merge(test_gap_df, submission_melted, on=['PID', 'Nutrient'], how='left')


In [ ]:
soil_depth = 20  # cm

# Calculate the Available_Nutrients_in_kg_ha
nutrient_df['Available_Nutrients_in_kg_ha'] = (nutrient_df['Available_Nutrients_in_ppm']
                                               * soil_depth * nutrient_df['BulkDensity'] * 0.1)

In [ ]:
nutrient_df.head()

In [ ]:
nutrient_df["Gap"] = nutrient_df["Required"] - nutrient_df["Available_Nutrients_in_kg_ha"]

In [ ]:
nutrient_df['ID'] = nutrient_df['PID'] + "_" + nutrient_df['Nutrient']
nutrient_df = nutrient_df[['ID', 'Gap']]
nutrient_df.head()

If a value is negative it means there is excess of that nutrient in the soil already and the farmer does not need to add any more. If the value is positive then the farmer needs to add those nutrients to the soil.

In [ ]:
nutrient_df.to_csv('submission.csv', index=False)
print("Submission file saved as submission.csv")

### **Next Steps**
- Experiment with different models (e.g., XGBoost, LightGBM).
- Tune hyperparameters to improve performance.
- Try feature engineering to enhance the dataset.

Happy Learning! 🎯